# Transmitting CAN Message to RAV4, copied from captured CAN Data

## Uninstall, Get update and Reinstall Panda

In [ ]:
!pip uninstall pandacan -y
!cd ..; git pull; python setup.py install

In [ ]:
import binascii
import bitstring
import time
import datetime
import serial
import csv
import numpy as np
import matplotlib.pyplot as plt

import random

import pandas as pd # Note that this is not commai Panda, but Database Pandas
import cantools 
import matplotlib.animation as animation
from matplotlib import style
import uuid

## Import CommaAI Panda package

In [ ]:
from panda import Panda #Import Comma AI Panda
__vehicleName__ = 'Rav4'
panda = Panda()

## Copy the required messages from an existing CAN Data CSV File

In [ ]:
import struct

## Filename of an already recorded data from where we will read messages to transmit
Filename = '2019-10-25-12-46-30-126677_8f806f3c-159e-4096-8552-b0aa7a0a681d_CAN_Message_Rav4.csv'
can_data = pd.read_csv(Filename)# read in the data

# We just want to send Message Type 466 and 467
MSG_467 = can_data[(can_data['MessageID'] == 467) | (can_data['MessageID'] == 466)]
Bus467 = MSG_467['Bus']
Message467 = MSG_467['Message']

MSG_467 = MSG_467.drop(columns="Time")
MSG_467 = MSG_467.drop(columns="MessageLength")

MSG_467['XXX'] = 61932

MSG_467 = MSG_467[['MessageID', 'XXX','Message', 'Bus']]

print(MSG_467)
TUPLEMsg = [tuple(x) for x in MSG_467.to_records(index=False)]

LENGTH = len(TUPLEMsg)

## Now we are ready to send messages to RAV4 using _can\_send\_many()_ function

In [ ]:
for i in range(0, LENGTH):
    print('sending')
    (msgID, XXX, Message, Bus) = TUPLEMsg[i]
    byteMessage = bytes(bytearray.fromhex(Message))
    byteMsg = bytearray(byteMessage)
    msgTuple = (msgID, XXX, byteMsg, Bus)
    msg = [msgTuple]
    print(msg)
    panda.can_send_many(msg)
    time.sleep(0.01)

#### But the above code sends message after reading from a file that has a captured data.
#### Below, I am providing a way to send message after manually constructing them

## Construct and send message via cantools

In [10]:
import cantools
import can
from pprint import pprint
db = cantools.database.load_file('newToyotacode.dbc')
example_message = db.get_message_by_frame_id(467)
print(example_message)
for i in range(0, len(example_message.signals)):
    print(example_message.signals[i])

message('PCM_CRUISE_2', 0x1d3, False, 8, None)
signal('MAIN_ON', 15, 1, 'big_endian', False, None, 1, 0, 0, 1, 'None', False, None, None, None)
signal('LOW_SPEED_LOCKOUT', 14, 2, 'big_endian', False, None, 1, 0, 0, 3, 'kph', False, None, {2: 'low speed locked', 1: 'ok'}, 'in low speed lockout, system would always disengage below 28mph')
signal('SET_SPEED', 23, 8, 'big_endian', False, None, 1, 0, 0, 255, 'kph', False, None, None, '43 kph are shown as 28mph, so conversion isnt perfect')
signal('CHECKSUM', 63, 8, 'big_endian', False, None, 1, 0, 0, 255, 'None', False, None, None, None)


In [21]:
for i in range(0, 2000):
    print('sending')
    main_on = 1
    low_speed_lockout = 1
    set_speed = 50
    messageId = 467
    messageLen = 5
    IDH = messageID & 0x0FF
    IDL = d>>8 & 0x0FF
    
    checksum = IDH + IDL + messageLen
    
    data = example_message.encode({'MAIN_ON': 1, 'LOW_SPEED_LOCKOUT': 1, 'SET_SPEED': 44, 'CHECKSUM':2 })
    (msgID, XXX, Message, Bus) = TUPLEMsg[i]
    byteMessage = bytes(bytearray.fromhex(Message))
    byteMsg = bytearray(byteMessage)
    msgTuple = (msgID, XXX, byteMsg, Bus)
    msg = [msgTuple]
    print(msg)
    panda.can_send_many(msg)
    time.sleep(0.01)

sending


NameError: name 'TUPLEMsg' is not defined

## Decoding an existing message using cantools

In [17]:
messageID = 740
message = "d8000000c3" # An existing message
byteMsg = bytes(bytearray.fromhex(message))
msgLen = len(byteMsg)

db.decode_message(messageID, byteMsg)

{'SET_ME_1': 1,
 'COUNTER': 44,
 'STEER_REQUEST': 0,
 'STEER_TORQUE_CMD': 0,
 'LKA_STATE': 0,
 'CHECKSUM': 195}


## Close the connection to Panda

In [ ]:
panda.close()

In [100]:
d = 467
h=hex(d)
print(h)

0x1d3


In [104]:
print(hex(d & 0x0FF))
print(hex(d>>8 & 0x0FF))

0xd3
0x1


SyntaxError: invalid syntax (<ipython-input-103-3999db33c645>, line 1)

In [ ]:
steer = (random.randint(0, 2) % 2 == 0)
steer_req = (random.randint(0, 2) % 2 == 0)
raw_cnt = random.randint(1, 65536)
def create_steer_command(packer, steer, steer_req, raw_cnt):
    """Creates a CAN message for the Toyota Steer Command."""
    values = {
        "STEER_REQUEST": steer_req,
        "STEER_TORQUE_CMD": steer,
        "COUNTER": raw_cnt,
        "SET_ME_1": 1,
    }
    return packer.make_can_msg("STEERING_LKA", 0, values)

In [ ]:
dbc_name = 'newToyotacode.dbc'
dbc_name = dbc_name.encode('utf8')